In [2]:
2+2

4

To find the space of possible honest $d\in \{0, 1\}^{2t}$ distributions, we use server privacy against malicious clients.

This is without accounting for (c, d)-pairs in the honest realm as special cases (here much more is known about **each** individual pair, rather then weaker constraints on the entire distribution $C^i$ $D^w$ that follow from correctness against malicious clients and servers respectively.)

In [1]:
# Model client correctness against malicious servers. Expected to have the biggest solution-liminiting impact
# generate the variable set

import itertools
import random

    
def d_at_c(d, c):
    return tuple(d[i][c[i]] for i, x in enumerate(d))
    

In [2]:
# Checks that a given vector solves the required equations of type 1. The inequalities x_i >= 0 
# are not included. The vector is specified by 3 dictionaries, for i=3, 2, 1 respectively
def checkCSolution(C, l=3):
    # TODO:
    # 1. Verify that the di's have disjoint key names
    # 2. Verify the structure of each index.
    # 3. The values in the dictionary should be in Q, as we need check equality precisely.
    # Verify that the induced distributions are indeed valid C's
    for i in range(3):
        key_prefixes = {t1 for (t1, t2, t3) in C[i].keys()}
        for t1 in key_prefixes:
            first_pref = (t1, (0, 0, 0))
            sum_first = sum(C[i][first_pref +  (pair, )] for pair in itertools.product(range(2), repeat=2))
            for t2 in itertools.product(range(2), repeat = 3):
                sum_cur = sum(C[i][(t1, t2) + (pair, )] for pair in itertools.product(range(2), repeat=2))
                if (sum_left - sum_first != 0):
                    return false
    for d in itertools.product(itertools.product(range(2), repeat = 2), repeat = 3):
        # dependencies within i=3
        sum_c3 = sum(C[3][(c3, d_at_c(d, c3), pair)] for (c3, pair) in itertools.product(C[3], itertools.product(range(2), repeat=2)))
        if (sum_c3 != 1):
            return false
        # dependencies of i=2 on i=2, 3
        c3_00_01 = sum(C[3][(c3, d_at_c(d, c3), (0, 0))] + C[3][(c3, d_at_c(d, c3), (0, 1))] for c3 in C[3])
        c2_00_01 = sum(C[2][(c2, d_at_c(d, c2), (0, 0))] + C[2][(c2, d_at_c(d, c2), (0, 1))] for c2 in C[2])
        
        if (c3_00_01 != c2_00_01):
            return false
        c3_10_11 = sum(C[3][(c3, d_at_c(d, c3), (1, 0))] + C[3][(c3, d_at_c(d, c3), (1, 1))] for c3 in C[3])
        c2_10_11 = sum(C[2][(c2, d_at_c(d, c2), (1, 0))] + C[2][(c2, d_at_c(d, c2), (1, 1))] for c2 in C[2])
        
        if (c3_10_11 - c2_10_11 != 0):
            return false
        
        # dependencies of i=1 on i=1, 2, 3
        c1_00_01 = sum(C[1][(c1, d_at_c(d, c1), (0, 0))] + C[1][(c1, d_at_c(d, c1), (0, 1))] for c1 in C[1])
        c3_00_10 = sum(C[3][(c3, d_at_c(d, c3), (0, 0))] + C[3][(c3, d_at_c(d, c3), (1, 0))] for c3 in C[3])
        
        if (c1_00_01 - c3_00_10 != 0):
            return false
        
        c1_00_10 = sum(C[1][(c1, d_at_c(d, c1), (0, 0))] + C[1][(c1, d_at_c(d, c1), (1, 0))] for c1 in C[1])
        c2_00_10 = sum(C[2][(c2, d_at_c(d, c2), (0, 0))] + C[2][(c2, d_at_c(d, c2), (1, 0))] for c2 in C[2])
        
        if (c1_00_10 - c2_00_10 != 0):
            return false
         
        c1_01_11 = sum(C[1][(c1, d_at_c(d, c1), (0, 1))] + C[1][(c1, d_at_c(d, c1), (1, 1))] for c1 in C[1])
        c2_01_11 = sum(C[2][(c2, d_at_c(d, c2), (0, 1))] + C[2][(c2, d_at_c(d, c2), (1, 1))] for c2 in C[2])
                
        if (c1_01_11 - c2_01_11 != 0):
            return false
        

In [11]:
# The dictionaries here specify only the first coordinate.
# Here we assume that the C[i]'s are disjoint. This is wlog. as honest correctness
# implies it (this is the only observation based on honest correctness that we are utilizing).
# Here the C[i]'s are lists of c's

def genClCorrEq(C):
    cur_ind = 0
    indices = {}
    right_hand_side = []
    for i in range(3):
        for c in C[i]:
            for d in itertools.product(range(2), repeat = 3):
                for pair in itertools.product(range(2), repeat = 2):
                    indices[(c, d, pair)] = cur_ind
                    cur_ind = cur_ind + 1
                    
    row_size = sum(len(C[i]) for i in range(3))
    rows = []
    # equality constraints                
    for i in range(3):
        for c in C[i]:
            for dv in itertools.product(range(2), repeat = 3):
                if dv != (0, 0, 0):
                    cur_row = [QQ(0) for i in range(row_size)]
                    for pair in itertools.product(range(2), 2):
                        cur_row[indices[(c, (0, 0, 0), pair)]] = QQ(1)
                        cur_row[indices[(c, dv, pair)]] = QQ(-1)
                    rows.append(cur_row)
                    right_hand_side.append(QQ(0))
            
            
    for d in itertools.product(itertools.product(range(2), repeat = 2), repeat = 3):
        cur_row = [QQ(0) for i in range(row_size)]
        # dependencies within i=3
        for c3 in C[3]:
            for pair in itertools.product(range(2), repeat = 2):
                cur_row[indices[(c3, d_at_c(d, c3), pair)]] = QQ(1)
        rows.append(cur_row)     
        right_hand_size.append(QQ(1))
                                
        # dependencies of i=2 on i=2, 3
        cur_row = [QQ(0) for i in range(row_size)]
        for c3 in C[3]:
            cur_row[indices[(c3, d_at_c(d, c3), (0, 0))]] = QQ(1)
            cur_row[indices[(c3, d_at_c(d, c3), (0, 1))]] = QQ(1)
        for c2 in C[2]:
            cur_row[indices[(c2, d_at_c(d, c2), (0, 0))]] = QQ(-1)
            cur_row[indices[(c2, d_at_c(d, c2), (0, 1))]] = QQ(-1)
        rows.append(cur_row)
        right_hand.append(QQ(0))            
                    
        cur_row = [QQ(0) for i in range(row_size)]
        for c3 in C[3]:
            cur_row[indices[(c3, d_at_c(d, c3), (1, 0))]] = QQ(1)
            cur_row[indices[(c3, d_at_c(d, c3), (1, 1))]] = QQ(1)
        for c2 in C[2]:
            cur_row[indices[(c2, d_at_c(d, c2), (1, 0))]] = QQ(-1)
            cur_row[indices[(c2, d_at_c(d, c2), (1, 1))]] = QQ(-1)
        rows.append(cur_row)
        right_hand.append(QQ(0))
                    
        # dependencies of i=1 on i=1, 2, 3
        cur_row = [QQ(0) for i in range(row_size)]
        for c1 in C[1]:
            cur_row[indices[(c3, d_at_c(d, c3), (0, 0))]] = QQ(1)
            cur_row[indices[(c3, d_at_c(d, c3), (0, 1))]] = QQ(1)
        for c3 in C[3]:
            cur_row[indices[(c2, d_at_c(d, c2), (0, 0))]] = QQ(-1)
            cur_row[indices[(c2, d_at_c(d, c2), (1, 0))]] = QQ(-1)
        rows.append(cur_row)
        right_hand.append(QQ(0))

        
        cur_row = [QQ(0) for i in range(row_size)]
        for c1 in C[1]:
            cur_row[indices[(c3, d_at_c(d, c3), (0, 0))]] = QQ(1)
            cur_row[indices[(c3, d_at_c(d, c3), (1, 0))]] = QQ(1)
        for c2 in C[2]:
            cur_row[indices[(c2, d_at_c(d, c2), (0, 0))]] = QQ(-1)
            cur_row[indices[(c2, d_at_c(d, c2), (1, 0))]] = QQ(-1)
        rows.append(cur_row)
        right_hand.append(QQ(0))
                    
        
        cur_row = [QQ(0) for i in range(row_size)]
        for c1 in C[1]:
            cur_row[indices[(c3, d_at_c(d, c3), (0, 1))]] = QQ(1)
            cur_row[indices[(c3, d_at_c(d, c3), (1, 1))]] = QQ(1)
        for c2 in C[2]:
            cur_row[indices[(c2, d_at_c(d, c2), (0, 1))]] = QQ(-1)
            cur_row[indices[(c2, d_at_c(d, c2), (1, 1))]] = QQ(-1)
        rows.append(cur_row)
        right_hand.append(QQ(0))
                    
    # Check matrix rank
    m = matrix(QQ, rows)
    print 'equations matrix = ', m
    return m.rank()               
    # Check for slovability               

def GenRandomC():
    C = [[], [], []]
    while True:                
        for c in itertools.product(range(2), repeat = 3):             
            r = random.randint(0, 3)                
            if (r != 3):
                C[r].append(c)
        if (len(C[0]) > 0) and (len(C[1]) > 0) and (len(C[2]) > 0):
            return C        

# main code
C = GenRandomC()
print 'C[0] =', C[0]
print 'C[1] =', C[1]
print 'C[2] =', C[2]
eq_rank = genClCorrEq(C)
print 'rank = ', eq_rank

C[0] = [(0, 0, 0), (1, 0, 0), (1, 0, 1)]
C[1] = [(0, 0, 1), (1, 1, 1)]
C[2] = [(0, 1, 1)]


NameError: global name 'QQ' is not defined

In [4]:
{i**2 for i in [-1, 0, 1]}

{0, 1}